In [28]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import numpy as np
import os

In [29]:
def create_connection(db_file: str) -> sqlite3.Connection:

    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [30]:
def general_data_insert(conn: sqlite3.Connection, general_data: pd.DataFrame) -> None:

    sql = ''' INSERT INTO General_Data(
              Match_id,
              Date, 
              Blue_Team, 
              Red_Team, 
              Tournament, 
              Time, 
              Game_Version)
              VALUES(?, ?, ?, ?, ?, ?, ?) '''
    cur = conn.cursor()
    cur.execute(sql, general_data)
    conn.commit()
    return cur.lastrowid

In [45]:
def players_stats_insert(conn: sqlite3.Connection, players_data: pd.DataFrame) -> None:

    sql = ''' INSERT INTO Players_Stats(
              Player_Nickname,
              KDA,
              CS,
              Damage_Distribution,
              Gold_Distribution,
              Champion,
              Role,
              Match_id,
              Match_count)
              VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?) '''
    cur = conn.cursor()
    cur.execute(sql, players_data)
    conn.commit()
    return cur.lastrowid

In [60]:
def team_stats_insert(conn: sqlite3.Connection, teams_data: pd.DataFrame) -> None:

    sql =  """
            Insert into Team_Stats(
              Club_Name,
              Result,
              Kills,
              First_Blood,
              Towers,
              First_Tower,
              Dragons,
              Barons,
              Gold,
              Ban_1,
              Ban_2,
              Ban_3,
              Ban_4,
              Ban_5,
              Pick_1,
              Pick_2,
              Pick_3,
              Pick_4,
              Pick_5,
              match_id,
              match_count
            )   
            VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
          """
    cur = conn.cursor()
    cur.execute(sql, teams_data)
    conn.commit()
    return cur.lastrowid

In [58]:
path = os.path.normpath(os.getcwd() + os.sep + os.pardir)
conn = create_connection(os.path.join(path, "LEAGUE_PRO_GAMES_ANALYSIS/data", "league.db"))
sqlite3.register_adapter(np.int64, lambda val: int(val))

In [56]:
def string_to_list(string: str) -> list:
    return string.replace('[', '').replace(']', '').replace("'", '').split(',')

In [55]:
# General data insert
for i in range(5):
    df_general = pd.read_csv(os.path.join(path, "LEAGUE_PRO_GAMES_ANALYSIS/data/raw", f"general_data_{i}.csv"), index_col=0)
    for i in range(len(df_general)):
        general_data_insert(conn, df_general.iloc[i])

In [61]:
# Team stats insert
for i in range(5):
    df = pd.read_csv(os.path.join(path, "LEAGUE_PRO_GAMES_ANALYSIS/data/raw", f"right_team_stats_{i}.csv"), index_col=0)
    for i in range(len(df)):
        bans = string_to_list(df['Bans'][i])
        picks = string_to_list(df['Picks'][i])
        team_stats_insert(conn, [df['Name'][i], df['Result'][i], df['Kills'][i], df['First_Blood'][i], df['Towers'][i],
       df['First_Tower'][i], df['Dragons'][i], df['Barons'][i], df['Gold'][i], bans[0], bans[1], bans[2], bans[3], bans[4], picks[0], picks[1], picks[2], picks[3], picks[4], df['Match_id'][i],
       df['Match_count'][i]])

ParserError: Error tokenizing data. C error: Expected 14 fields in line 1711, saw 21


In [43]:
# Team stats insert
for i in range(5):
    df_team_stats = pd.read_csv(os.path.join(path, "LEAGUE_PRO_GAMES_ANALYSIS/data/raw", f"left_team_stats_{i}.csv"), index_col=0)
    for i in range(len(df_team_stats)):
        general_data_insert(conn, df_team_stats.iloc[i])

ProgrammingError: Incorrect number of bindings supplied. The current statement uses 7, and there are 13 supplied.

In [54]:
# Player stats insert
for i in range(6):
    df_player_stats = pd.read_csv(os.path.join(path, "LEAGUE_PRO_GAMES_ANALYSIS/data/raw", f"players_stats_{i}.csv"), index_col=0)
    for i in range(len(df_player_stats)):
        players_stats_insert(conn, df_player_stats.iloc[i])

In [49]:
def select_all_tasks(conn: sqlite3.Connection):

    cur = conn.cursor()
    cur.execute("SELECT * FROM Players_Stats")

    rows = cur.fetchall()

    for row in rows:
        print(row)

In [50]:
select_all_tasks(conn)

("['Tholem']", "['2/3/7']", "['239']", '18.2%', '18.9%', 'Gragas', 'Top', 39121, 0)
("['chef']", "['3/3/10']", "['186']", '25.8%', '19.6%', 'Hecarim', 'Jungle', 39121, 0)
("['Avarosa']", "['5/4/7']", "['261']", '24.3%', '22.8%', 'Ahri', 'Mid', 39121, 0)
("['Pako']", "['7/2/7']", "['324']", '24.4%', '26.1%', 'Jinx', 'Adc', 39121, 0)
("['monkaS']", "['0/5/15']", "['37']", '7.3%', '12.6%', 'Leona', 'Support', 39121, 0)
("['Naak', 'Nako']", "['5/2/4']", "['347']", '21.2%', '25.7%', 'Yasuo', 'Top', 39121, 0)
("['361efe']", "['0/5/6']", "['178']", '6.1%', '15.4%', 'Xin', 'Jungle', 39121, 0)
("['Kunduz']", "['5/3/9']", "['246']", '26.4%', '21.4%', 'LeBlanc', 'Mid', 39121, 0)
("['Ruep']", "['7/4/6']", "['318']", '38.3%', '24.4%', 'Xayah', 'Adc', 39121, 0)
("['Carry']", "['0/3/17']", "['25']", '8.1%', '13.2%', 'Rakan', 'Support', 39121, 0)
("['Naak', 'Nako']", "['8/5/2']", "['243']", '28.6%', '26.1%', 'Akshan', 'Top', 39121, 1)
("['361efe']", "['2/4/4']", "['156']", '9.2%', '16%', 'Xin', 'Jungl

KeyboardInterrupt: 

In [39]:
type(df_general['Match_id'].iloc[0])

numpy.int64